# Quickstart guide

``imcascade`` is a non-parametric framework for fitting objects in astronomical images. This is accomplished by modelling the objects a series of Gaussians. For full details please read our paper here: *Put ArXiv link here*. What follows is a (very) brief introduction to the basic usage of ``imcascade``.

In [1]:
# This cell is hidden from documentation but the analytic profile and psf are calculated here using the astropy definitions
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling.functional_models import Sersic2D, Moffat2D
from astropy.convolution import convolve
grid = np.arange(0,150)
X,Y = np.meshgrid(grid,grid)
s2d = Sersic2D(r_eff = 5,n = 1.5, x_0 = 75.2, y_0 = 75.2)
im = s2d(X,Y)
im = im*100./np.sum(im)

grid2 = np.arange(0,31)
X,Y = np.meshgrid(grid2,grid2)
m2d = Moffat2D(gamma = 3,alpha = 3, x_0 = 15., y_0 = 15.)
psf = m2d(X,Y)
psf = psf/ np.sum(psf)

sci = convolve(im,psf) + np.random.normal(loc = 0, scale = 5e-2, size = im.shape)

In this short example we will fit an analytic, circular, Sersic profile with $n = 1.5$, $r_e = 5$ and total flux, $F = 100$, we have added purely Gaussian noise and convolved the profile with a Moffat PSF with $\alpha = 3$ and $\gamma = 3$

In a hidden cell I have intialized the cutout in the 2D array ``sci`` and the pixelized PSF saved in the 2D array ``psf``.

The ``InitializedFitter`` function is designed to take a pixelized science images (as 2D arrays or fits files) and help initalize a ``Fitter`` instance which will be used below to fit galaxies. This function is designed to help users get started using our experiences to help guide some of the decisions, which may not be applicable in all situations. For more details about these choices or other options please see [the in depth example](example.ipynb) for a longer discussion.

In [5]:
from imcascade.fitter import fitter_from_ASDF

In [2]:
from imcascade.fitter import initialize_fitter

fitter = initialize_fitter(sci,psf)

2021-05-31 15:02:56,839 - Fit PSF with 4 components
2021-05-31 15:02:56,842 - Widths: 1.2,1.81,5.28,2.93
2021-05-31 15:02:56,845 - Fluxes: 0.27,0.51,0.03,0.19
2021-05-31 15:02:56,877 - Using 9 components with logarithmically spaced widths to fit galaxy
2021-05-31 15:02:56,879 - 0.87, 1.37, 2.15, 3.39, 5.33, 8.38, 13.17, 20.71, 32.57
2021-05-31 15:02:56,881 - No mask was given, derriving one using sep
2021-05-31 15:02:56,929 - Using sep rms map to calculate pixel weights


This function uses the ``psf_fitter`` module to fit the given pixelized psf with a sum of Gaussians, which is required for our method. Next it estimates the effective radius and uses nine logarithmically spaced widths for the Gaussian components ranging from the PSF HWHM to $10\times r_e$. It then derrives pixel weights and masks using [sep](https://sep.readthedocs.io/en/v1.1.x/) (or the gain,exposure time and readnoise to calculate the rms). There are also options to use pre-calculated version of these if the user has them.

Now we will run our least-squares minimization routine

In [3]:
_ = fitter.run_ls_min()

2021-05-31 15:02:56,957 - Running least squares minimization
2021-05-31 15:03:56,071 - Finished least squares minimization


Now we will use our ``results`` submodule to help us analyze the results

In [4]:
from imcascade.results import ImcascadeResults
res_ls = ImcascadeResults(fitter)
res_ls.run_basic_analysis()

{'flux': 97.72826118464025,
 'r20': 1.9478206698732747,
 'r50': 4.860906958232164,
 'r80': 9.559362403779067,
 'r90': 12.515313905863412,
 'C80_20': 4.907722025765852,
 'C90_50': 2.5746869901856826}

We can see our fit matches the true profile pretty well!

Next we will explore the Posterior distribution using Dynesty, and our method of pre-rendered images to help speed up the run time.

In [5]:
post = fitter.run_dynesty(method = 'express')

2021-05-31 15:03:56,135 - Running dynesty using the express method
2021-05-31 15:03:56,137 - Setting up Express params
2021-05-31 15:03:56,141 - Parameters to be set for pre-rendered images:
2021-05-31 15:03:56,146 - 	 Galaxy Center: 76.34,75.26
2021-05-31 15:03:56,149 - 	 Axis Ratio: 1.00000
2021-05-31 15:03:56,154 - 	 PA: 1.76793
iter: 22465 | batch: 7 | bound: 241 | nc: 25 | ncall: 529304 | eff(%):  4.244 | loglstar: 35388.750 < 35396.458 < 35394.847 | logz: 35381.749 +/-  0.209 | stop:  0.973                               2021-05-31 15:27:24,837 - Finished running dynesty, calculating posterier


Now if we run the analysis again, we will have slightly different central values and error as the 16th-84th percentile range of the posterior.

In [6]:
res_dyn = ImcascadeResults(fitter)
res_dyn.run_basic_analysis()

{'flux': array([97.80483983,  2.10467908,  2.25823866]),
 'r20': array([1.94562589, 0.06065083, 0.05934397]),
 'r50': array([4.88723217, 0.13187   , 0.13756048]),
 'r80': array([9.52563547, 0.34671484, 0.41985052]),
 'r90': array([12.51954456,  0.60381429,  0.78604302]),
 'C80_20': array([4.90765941, 0.15687631, 0.18068263]),
 'C90_50': array([2.56555266, 0.09066102, 0.11327043])}